In [3]:
import fasttext
import os
import json
from utils import fixText, resetLabelLv2, resetLabelLv3

TEMP_DIR = './tmp/'

def getConfig():
    config = {}
    with open('./config.json', 'r') as f:
        s = f.read()
        config = json.loads(s)
    return config

# 读入数据集的json文件，处理成fasttext接口使用的"文本__label__标签"形式，以txt文件存储
def readDataSet(config):
    set1 = []
    with open(config['data_path'] + config['dataset_label_1_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            set1.append('__label__' + data_set[id]['label_level_1'] + ' ' + fixText(data_set[id]['text']))
    set2 = []
    with open(config['data_path'] + config['dataset_label_2_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            set2.append('__label__' + resetLabelLv2(data_set[id]['label_level_2']) + ' ' + fixText(data_set[id]['text']))
    set3 = []
    with open(config['data_path'] + config['dataset_label_3_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            set3.append('__label__' + resetLabelLv3(data_set[id]['label_level_3']) + ' ' + fixText(data_set[id]['text']))
        
    try:
        os.mkdir(TEMP_DIR)
    except:
        pass
    with open(TEMP_DIR + 'set1.txt', 'w') as f:
        for l in set1:
            f.write(l + '\n')
    with open(TEMP_DIR + 'set2.txt', 'w') as f:
        for l in set2:
            f.write(l + '\n')
    with open(TEMP_DIR + 'set3.txt', 'w') as f:
        for l in set3:
            f.write(l + '\n')

config = getConfig()
readDataSet(config)

In [4]:
model_label1 = fasttext.train_supervised(
    input = TEMP_DIR + 'set1.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)

Read 1M words
Number of words:  36184
Number of labels: 85
Progress: 100.0% words/sec/thread:  792217 lr:  0.000000 avg.loss:  1.996176 ETA:   0h 0m 0s


In [ ]:
model_label2 = fasttext.train_supervised(
    input = TEMP_DIR + 'set2.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)

In [ ]:
model_label3 = fasttext.train_supervised(
    input = TEMP_DIR + 'set3.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)

In [2]:
try:
    os.remove(TEMP_DIR + 'set1.txt')
    os.remove(TEMP_DIR + 'set2.txt')
    os.remove(TEMP_DIR + 'set3.txt')
    os.removedirs(TEMP_DIR)  
except:
    pass

In [5]:
with open(config['data_path'] + 'test_set.json', 'r') as f:
    content = json.loads(f.read())

total = 0
true_label_1 = 0
true_label_2 = 0
true_label_3 = 0

label_1_dict = {}
for id in content.keys():
    total += 1
    text = content[id]['text']
    text = fixText(text)
    
    label_1 = content[id]['label_level_1']
    # label_2 = resetLabelLv2(content[id]['label_level_2'])
    # label_3 = resetLabelLv3(content[id]['label_level_3'])

    if label_1 in label_1_dict:
        label_1_dict[label_1][0] += 1
    else:
        label_1_dict[label_1] = [1, 0]

    predict_1 = model_label1.predict(text)[0][0]
    # predict_2 = model_label2.predict(text)[0][0]
    # predict_3 = model_label3.predict(text)[0][0]

    if(predict_1.replace('__label__', '') == label_1):
        true_label_1 += 1
        label_1_dict[label_1][1] += 1
    # if(predict_2.replace('__label__', '') == label_2):
    #     true_label_2 += 1
    # if(predict_3.replace('__label__', '') == label_3):
    #     true_label_3 += 1
print('tag level 1 accurate: {}% ({}/{})'.format(true_label_1 * 100 / total, true_label_1, total))
# print('tag level 2 accurate: {}% ({}/{})'.format(true_tag2 * 100 / total, true_tag2, total))
# print('tag level 3 accurate: {}% ({}/{})'.format(true_tag3 * 100 / total, true_tag3, total))

tag level 1 accurate: 3.229846073571615% (1238/38330)


In [6]:
for label in label_1_dict:
    print(label, label_1_dict[label][1], "/", label_1_dict[label][0])

荔湾区政府 0 / 36058
市卫生健康委 312 / 343
市住房城乡建设局 45 / 71
市公安局 117 / 142
广州地铁集团 42 / 68
市规划和自然资源局 0 / 20
越秀区政府 183 / 290
番禺区政府 21 / 58
岭南商旅集团 0 / 1
市邮政管理局 15 / 30
市交通运输局 71 / 111
天河区政府 42 / 116
白云区政府 128 / 203
海珠区政府 53 / 156
市来穗人员服务管理局 0 / 11
穗康码工作专班 0 / 36
珠江实业集团 13 / 19
广州市税务局 39 / 58
市人力资源社会保障局 10 / 39
广州供电局有限公司 44 / 59
市排水公司 2 / 26
市建筑集团 0 / 3
市民政局 0 / 11
市国资委 0 / 5
黄埔区政府 0 / 44
广州市自来水公司 93 / 108
市文化广电旅游局 0 / 1
市林业园林局 0 / 2
花都区政府 0 / 17
增城区政府 0 / 23
广州市公共交通集团有限公司 0 / 13
从化区政府 0 / 4
市城市管理综合执法局 0 / 6
市教育局 8 / 23
中国移动广州分公司 0 / 4
12345热线管理机构 0 / 15
市财政局 0 / 1
市地方金融监管局 0 / 13
市医保局 0 / 12
南沙区政府 0 / 37
广州市烟草专卖局 0 / 3
市政务服务数据管理局 0 / 3
广州住房公积金管理中心 0 / 1
中国联通广州分公司 0 / 2
农业银行广东省分行 0 / 1
市水投集团 0 / 5
市港务局 0 / 1
市残联 0 / 1
中国电信广州分公司 0 / 6
市燃气集团 0 / 7
市市场监管局 0 / 5
广州交投集团 0 / 6
市司法局 0 / 3
广州商贸投资控股集团有限公司 0 / 1
广州环保投资集团 0 / 2
广州轻工工贸集团 0 / 5
市总工会 0 / 2
市新闻出版局（市版权局） 0 / 2
市气象局 0 / 2
市供销合作总社 0 / 2
广州日报 0 / 1
市城投集团 0 / 1
市水务局 0 / 1
中国广电广州网络股份有限公司 0 / 2
广州海事局 0 / 1
建设银行广东省分行 0 / 1
市商务局 0 / 3
广州市消费者委员会 0 / 1
越秀集团 

lr = 0.1 epoch = 25

标签数量

In [8]:
print('tag 1 num: ', len(model_label1.labels))
# print('tag 2 num: ', len(model_label2.labels))
# print('tag 3 num: ', len(model_label3.labels))

tag 2 num:  459
tag 3 num:  970
